<a href="https://colab.research.google.com/github/Raoina/Spectra-2-Image/blob/main/notebooks/processing_dataset/siware_avg_preprocessing_train_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook Summary

This notebook processed spectral data for training and testing a machine learning model. The following steps were performed:

1.  **Load Training Data**: The training data (`DL_FreshCS_train_Multiple_Refs.csv`) was loaded into a pandas DataFrame (`df`).
2.  **Clean Training Data**: The 'Scanner ID' column was dropped from the training DataFrame.
3.  **Handle Training Duplicates**: Duplicate rows in the training data based on 'Sample ID' were identified and then handled by averaging the values for each unique 'Sample ID', resulting in the `averaged_df`.
4.  **Add Sample Index (Train)**: A 'Sample_Index' column with ascending numbers was added to `averaged_df` and placed as the first column.
5.  **Split Training Data**: The `averaged_df` was split into `target_train_400` (containing 'Sample_Index', 'Moi', 'NDF', 'Starch') and `spectra_train_400` (containing 'Sample_Index' and the spectral data).
6.  **Save Processed Training Data**: The `averaged_df`, `target_train_400`, and `spectra_train_400` DataFrames were saved as CSV files in the `siware_data_avg_by_scannerID` folder in Google Drive.
7.  **Load Test Data**: The test data (`/content/DL_FreshCS_test_Multiple_Refs.csv`) was loaded into a pandas DataFrame (`df_test`).
8.  **Check Test Duplicates (Sample & Scanner)**: Duplicate rows in `df_test` based on 'Sample' and 'Scanner' were identified and counted.
9.  **Check Test Duplicates (Each Column)**: Duplicates in each individual column of `df_test` were checked.
10. **Check Test Nulls**: Null values in `df_test` were checked.
11. **Process Test Data (No Averaging)**: 'Scanner' and 'Sample' columns were dropped from `df_test`, and a 'Sample_Index' column with ascending numbers was added as the first column, as requested.
12. **Split Test Data**: The processed `df_test` was split into `target_test_400` (containing 'Sample_Index', 'Moi', 'NDF', 'Starch') and `spectra_test_400` (containing 'Sample_Index' and the spectral data).
13. **Save Processed Test Data**: The processed `df_test`, `target_test_400`, and `spectra_test_400` DataFrames were saved as CSV files with the "_no_avg" suffix in the `siware_data_avg_by_scannerID` folder in Google Drive.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd

file_path = '/content/drive/MyDrive/DL_FreshCS_train_Multiple_Refs.csv'

df = pd.read_csv(file_path)
display(df.head())

,Scanner ID,Sample ID,Moi,NDF,Starch,3921.568654,3935.185205,3948.801765,3962.418316,3976.034876,...,7284.857826,7298.474386,7312.090937,7325.707497,7339.324048,7352.940608,7366.557159,7380.173719,7393.790269,7407.406829
0,0,Fermented_#_519374,65.916589,34.34,37.86,7.942349,7.879306,7.818098,7.758485,7.708171,...,28.317286,28.983611,29.563827,30.078988,30.546206,30.978860,31.386878,31.777386,32.156782,32.532960
1,0,Fermented_#_519374,65.916589,34.34,37.86,7.419641,7.349567,7.286254,7.220075,7.152485,...,30.350159,31.063256,31.657065,32.168421,32.636662,33.087229,33.531162,33.970926,34.407003,34.843520
2,0,Fermented_#_519374,65.916589,34.34,37.86,7.505473,7.453863,7.412702,7.378596,7.353883,...,27.907187,28.491096,29.009951,29.480590,29.909929,30.302208,30.666498,31.019581,31.383385,31.778342
3,0,Fermented_#_519374,65.916589,34.34,37.86,7.501803,7.404250,7.300882,7.197936,7.111502,...,29.322861,29.997931,30.556176,31.034004,31.470861,31.893944,32.316505,32.741858,33.169309,33.600686
4,0,Fermented_#_519374,65.916589,34.34,37.86,7.249132,7.194527,7.142602,7.090535,7.044868,...,27.426496,28.029293,28.492752,28.871351,29.224750,29.601149,30.024162,30.489263,30.971840,31.443427


In [10]:
print(df.shape)

(8163, 262)


In [11]:
# Replace ['column_name_1', 'column_name_2'] with the actual names of the columns you want to drop
columns_to_drop = ['Scanner ID']

# Drop the columns
df.drop(columns=columns_to_drop, axis=1, inplace=True)

# Display the first few rows of the DataFrame after dropping columns
display(df.head())
print(df.shape)

,Sample ID,Moi,NDF,Starch,3921.568654,3935.185205,3948.801765,3962.418316,3976.034876,3989.651427,...,7284.857826,7298.474386,7312.090937,7325.707497,7339.324048,7352.940608,7366.557159,7380.173719,7393.790269,7407.406829
0,Fermented_#_519374,65.916589,34.34,37.86,7.942349,7.879306,7.818098,7.758485,7.708171,7.678701,...,28.317286,28.983611,29.563827,30.078988,30.546206,30.978860,31.386878,31.777386,32.156782,32.532960
1,Fermented_#_519374,65.916589,34.34,37.86,7.419641,7.349567,7.286254,7.220075,7.152485,7.093691,...,30.350159,31.063256,31.657065,32.168421,32.636662,33.087229,33.531162,33.970926,34.407003,34.843520
2,Fermented_#_519374,65.916589,34.34,37.86,7.505473,7.453863,7.412702,7.378596,7.353883,7.344609,...,27.907187,28.491096,29.009951,29.480590,29.909929,30.302208,30.666498,31.019581,31.383385,31.778342
3,Fermented_#_519374,65.916589,34.34,37.86,7.501803,7.404250,7.300882,7.197936,7.111502,7.058376,...,29.322861,29.997931,30.556176,31.034004,31.470861,31.893944,32.316505,32.741858,33.169309,33.600686
4,Fermented_#_519374,65.916589,34.34,37.86,7.249132,7.194527,7.142602,7.090535,7.044868,7.017365,...,27.426496,28.029293,28.492752,28.871351,29.224750,29.601149,30.024162,30.489263,30.971840,31.443427


(8163, 261)


In [12]:
# Calculate duplicates in 'Sample ID' column
duplicates = df[df.duplicated('Sample ID', keep=False)]


# Count the number of duplicate 'Sample ID' entries
num_duplicates = duplicates['Sample ID'].nunique()
print(f"\nNumber of duplicate 'Sample ID' entries: {num_duplicates}")

# Count the total rows involved in duplicates
total_duplicate_rows = len(duplicates)
print(f"Total rows involved in duplicate 'Sample ID' entries: {total_duplicate_rows}")


Number of duplicate 'Sample ID' entries: 783
Total rows involved in duplicate 'Sample ID' entries: 8162


In [21]:
# Group by 'Scanner ID' and calculate the mean of other numerical columns
averaged_df = df.groupby('Sample ID').mean()

# Reset the index to turn 'Scanner ID' back into a column
averaged_df.reset_index(inplace=True)

print("DataFrame with averaged values grouped by Scanner ID:")
display(averaged_df.head())
print(averaged_df.shape)

DataFrame with averaged values grouped by Scanner ID:


,Sample ID,Moi,NDF,Starch,3921.568654,3935.185205,3948.801765,3962.418316,3976.034876,3989.651427,...,7284.857826,7298.474386,7312.090937,7325.707497,7339.324048,7352.940608,7366.557159,7380.173719,7393.790269,7407.406829
0,Fermented_#_480926,60.823286,35.56,37.20,7.729080,7.622415,7.546208,7.481664,7.423155,7.377601,...,34.616084,35.288379,35.845357,36.327769,36.777222,37.228529,37.695179,38.173677,38.652607,39.125314
1,Fermented_#_480927,50.485837,29.28,45.84,9.720940,9.584006,9.485654,9.401768,9.326291,9.270802,...,42.449703,43.100897,43.648566,44.136925,44.609373,45.097435,45.608556,46.131176,46.646221,47.142563
2,Fermented_#_480929,63.115711,41.44,33.25,7.551404,7.444440,7.367629,7.300275,7.237015,7.186719,...,34.907780,35.560699,36.103651,36.578961,37.026430,37.475247,37.932452,38.389652,38.834516,39.264430
3,Fermented_#_480932,65.419594,39.32,32.91,6.526686,6.434280,6.366056,6.301643,6.235700,6.175937,...,27.640183,28.252847,28.764077,29.212927,29.634762,30.056096,30.485576,30.919745,31.351121,31.777759
4,Fermented_#_480933,62.917114,31.35,41.55,6.840954,6.757292,6.697071,6.637614,6.573048,6.512227,...,28.069196,28.715024,29.254844,29.724470,30.160514,30.594707,31.040131,31.494608,31.948344,32.395361


(784, 261)


In [22]:
# Calculate duplicates in 'Sample ID' column of averaged_df
duplicates_averaged = averaged_df[averaged_df.duplicated('Sample ID', keep=False)]

# Display the duplicates
print("Duplicate entries based on 'Sample ID' in averaged_df:")
display(duplicates_averaged)

# Count the number of duplicate 'Sample ID' entries
num_duplicates_averaged = duplicates_averaged['Sample ID'].nunique()
print(f"\nNumber of duplicate 'Sample ID' entries in averaged_df: {num_duplicates_averaged}")

# Count the total rows involved in duplicates
total_duplicate_rows_averaged = len(duplicates_averaged)
print(f"Total rows involved in duplicate 'Sample ID' entries in averaged_df: {total_duplicate_rows_averaged}")

Duplicate entries based on 'Sample ID' in averaged_df:


,Sample ID,Moi,NDF,Starch,3921.568654,3935.185205,3948.801765,3962.418316,3976.034876,3989.651427,...,7284.857826,7298.474386,7312.090937,7325.707497,7339.324048,7352.940608,7366.557159,7380.173719,7393.790269,7407.406829



Number of duplicate 'Sample ID' entries in averaged_df: 0
Total rows involved in duplicate 'Sample ID' entries in averaged_df: 0


In [23]:
# Check for null values in all columns of averaged_df
null_counts = averaged_df.isnull().sum()

# Display the columns with null values and their counts
print("Null values per column in averaged_df:")
print(null_counts[null_counts > 0])

Null values per column in averaged_df:
Series([], dtype: int64)


In [24]:
# Create a new column with ascending numbers starting from 1
averaged_df['Sample_Index'] = range(1, len(averaged_df) + 1)

# Drop the 'Sample ID' column
averaged_df.drop('Sample ID', axis=1, inplace=True)

# Get the list of columns
cols = averaged_df.columns.tolist()

# Move 'Sample_Index' to the front of the list
cols.insert(0, cols.pop(cols.index('Sample_Index')))

# Reindex the DataFrame with the new column order
averaged_df = averaged_df[cols]

# Display the first few rows with the new column
display(averaged_df.head())

,Sample_Index,Moi,NDF,Starch,3921.568654,3935.185205,3948.801765,3962.418316,3976.034876,3989.651427,...,7284.857826,7298.474386,7312.090937,7325.707497,7339.324048,7352.940608,7366.557159,7380.173719,7393.790269,7407.406829
0,1,60.823286,35.56,37.20,7.729080,7.622415,7.546208,7.481664,7.423155,7.377601,...,34.616084,35.288379,35.845357,36.327769,36.777222,37.228529,37.695179,38.173677,38.652607,39.125314
1,2,50.485837,29.28,45.84,9.720940,9.584006,9.485654,9.401768,9.326291,9.270802,...,42.449703,43.100897,43.648566,44.136925,44.609373,45.097435,45.608556,46.131176,46.646221,47.142563
2,3,63.115711,41.44,33.25,7.551404,7.444440,7.367629,7.300275,7.237015,7.186719,...,34.907780,35.560699,36.103651,36.578961,37.026430,37.475247,37.932452,38.389652,38.834516,39.264430
3,4,65.419594,39.32,32.91,6.526686,6.434280,6.366056,6.301643,6.235700,6.175937,...,27.640183,28.252847,28.764077,29.212927,29.634762,30.056096,30.485576,30.919745,31.351121,31.777759
4,5,62.917114,31.35,41.55,6.840954,6.757292,6.697071,6.637614,6.573048,6.512227,...,28.069196,28.715024,29.254844,29.724470,30.160514,30.594707,31.040131,31.494608,31.948344,32.395361


In [25]:
# Create target_train_400 DataFrame
target_train_400 = averaged_df[['Sample_Index', 'Moi', 'NDF', 'Starch']].copy()

# Create spectra_train_400 DataFrame
# Select all columns except 'Moi', 'NDF', and 'Starch'
spectra_train_400 = averaged_df.drop(['Moi', 'NDF', 'Starch'], axis=1).copy()

print("Target DataFrame (target_train_400):")
display(target_train_400.head())

print("\nSpectra DataFrame (spectra_train_400):")
display(spectra_train_400.head())

Target DataFrame (target_train_400):


,Sample_Index,Moi,NDF,Starch
0,1,60.823286,35.56,37.20
1,2,50.485837,29.28,45.84
2,3,63.115711,41.44,33.25
3,4,65.419594,39.32,32.91
4,5,62.917114,31.35,41.55



Spectra DataFrame (spectra_train_400):


,Sample_Index,3921.568654,3935.185205,3948.801765,3962.418316,3976.034876,3989.651427,4003.267987,4016.884537,4030.501097,...,7284.857826,7298.474386,7312.090937,7325.707497,7339.324048,7352.940608,7366.557159,7380.173719,7393.790269,7407.406829
0,1,7.729080,7.622415,7.546208,7.481664,7.423155,7.377601,7.358989,7.381517,7.454154,...,34.616084,35.288379,35.845357,36.327769,36.777222,37.228529,37.695179,38.173677,38.652607,39.125314
1,2,9.720940,9.584006,9.485654,9.401768,9.326291,9.270802,9.256881,9.306069,9.431605,...,42.449703,43.100897,43.648566,44.136925,44.609373,45.097435,45.608556,46.131176,46.646221,47.142563
2,3,7.551404,7.444440,7.367629,7.300275,7.237015,7.186719,7.165871,7.190576,7.270485,...,34.907780,35.560699,36.103651,36.578961,37.026430,37.475247,37.932452,38.389652,38.834516,39.264430
3,4,6.526686,6.434280,6.366056,6.301643,6.235700,6.175937,6.136361,6.129914,6.163474,...,27.640183,28.252847,28.764077,29.212927,29.634762,30.056096,30.485576,30.919745,31.351121,31.777759
4,5,6.840954,6.757292,6.697071,6.637614,6.573048,6.512227,6.471038,6.464014,6.498781,...,28.069196,28.715024,29.254844,29.724470,30.160514,30.594707,31.040131,31.494608,31.948344,32.395361


In [28]:
import os

# Define the folder path in Google Drive
folder_path = '/content/drive/MyDrive/siware_data_avg_by_scannerID'

# Create the folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

# Define the file paths for saving the DataFrames
averaged_df_path = os.path.join(folder_path, 'train_700.csv')
target_train_400_path = os.path.join(folder_path, 'target_train_700.csv')
spectra_train_400_path = os.path.join(folder_path, 'spectra_train_700.csv')

# Save the DataFrames to CSV files
averaged_df.to_csv(averaged_df_path, index=False)
target_train_400.to_csv(target_train_400_path, index=False)
spectra_train_400.to_csv(spectra_train_400_path, index=False)

print(f"DataFrames saved to: {folder_path}")

DataFrames saved to: /content/drive/MyDrive/siware_data_avg_by_scannerID


# Test

In [ ]:
df_test.head()

,Scanner,Sample,Moi,NDF,Starch,3921.568654,3935.18188,3948.795106,3962.408332,3976.021558,...,7284.03545,7297.648676,7311.261902,7324.875128,7338.488353,7352.101579,7365.714805,7379.328031,7392.941257,7406.554483
0,22030077,1,57.02,37.67,37.84,7.209724,7.202985,7.208119,7.222696,7.258278,...,38.932550,39.595226,40.219869,40.823632,41.396258,41.917661,42.375544,42.786740,43.177964,43.578847
1,22030077,1,57.02,37.67,37.84,6.802202,6.786788,6.780681,6.782455,6.805124,...,37.986222,38.679829,39.260441,39.754916,40.186044,40.582608,40.971723,41.394751,41.882680,42.451031
2,22030077,1,57.02,37.67,37.84,7.890556,7.886369,7.891463,7.899888,7.926916,...,40.293261,40.944037,41.545459,42.118273,42.643149,43.082208,43.410055,43.651383,43.865255,44.137235
3,22030077,1,57.02,37.67,37.84,7.741266,7.740978,7.762811,7.796629,7.849472,...,39.061605,39.623743,40.143299,40.666132,41.198445,41.715807,42.192246,42.641311,43.095348,43.595889
4,22030077,1,57.02,37.67,37.84,6.642168,6.647871,6.665882,6.686998,6.720952,...,37.307641,38.113560,38.788978,39.330331,39.747478,40.085455,40.398576,40.749179,41.170187,41.662778


In [ ]:
df_test.shape

(432, 262)

In [ ]:
# Check for duplicates based on 'Sample' and 'Scanner' columns
duplicates_test = df_test[df_test.duplicated(subset=['Sample', 'Scanner'], keep=False)]

num_duplicates_test_pairs = duplicates_test.groupby(['Sample', 'Scanner']).ngroups
print(f"\nNumber of duplicate 'Sample' and 'Scanner' pairs in df_test: {num_duplicates_test_pairs}")

# Count the total rows involved in duplicates
total_duplicate_rows_test = len(duplicates_test)
print(f"Total rows involved in duplicate 'Sample' and 'Scanner' entries in df_test: {total_duplicate_rows_test}")


Number of duplicate 'Sample' and 'Scanner' pairs in df_test: 72
Total rows involved in duplicate 'Sample' and 'Scanner' entries in df_test: 432


In [ ]:
print("Checking for duplicates in each column of df_test:")
for col in df_test.columns:
    duplicates_count = df_test[col].duplicated().sum()
    if duplicates_count > 0:
        print(f"Column '{col}': {duplicates_count} duplicates")

Checking for duplicates in each column of df_test:
Column 'Scanner': 420 duplicates
Column 'Sample': 426 duplicates
Column 'Moi': 426 duplicates
Column 'NDF': 426 duplicates
Column 'Starch': 426 duplicates


no need average spectra test data because it will become too small

In [ ]:
# Check for null values in all columns of df_test
null_counts_test = df_test.isnull().sum()

# Display the columns with null values and their counts
print("Null values per column in df_test:")
print(null_counts_test[null_counts_test > 0])

Null values per column in df_test:
Series([], dtype: int64)


In [ ]:
# Drop 'Scanner' and 'Sample' columns
df_test.drop(['Scanner', 'Sample'], axis=1, inplace=True)

# Create a new column with ascending numbers starting from 1
df_test['Sample_Index'] = range(1, len(df_test) + 1)

# Get the list of columns
cols_test = df_test.columns.tolist()

# Move 'Sample_Index' to the front of the list
cols_test.insert(0, cols_test.pop(cols_test.index('Sample_Index')))

# Reindex the DataFrame with the new column order
df_test = df_test[cols_test]

# Display the first few rows with the new column
display(df_test.head())

,Sample_Index,Moi,NDF,Starch,3921.568654,3935.18188,3948.795106,3962.408332,3976.021558,3989.634784,...,7284.03545,7297.648676,7311.261902,7324.875128,7338.488353,7352.101579,7365.714805,7379.328031,7392.941257,7406.554483
0,1,57.02,37.67,37.84,7.209724,7.202985,7.208119,7.222696,7.258278,7.331684,...,38.932550,39.595226,40.219869,40.823632,41.396258,41.917661,42.375544,42.786740,43.177964,43.578847
1,2,57.02,37.67,37.84,6.802202,6.786788,6.780681,6.782455,6.805124,6.866235,...,37.986222,38.679829,39.260441,39.754916,40.186044,40.582608,40.971723,41.394751,41.882680,42.451031
2,3,57.02,37.67,37.84,7.890556,7.886369,7.891463,7.899888,7.926916,7.996553,...,40.293261,40.944037,41.545459,42.118273,42.643149,43.082208,43.410055,43.651383,43.865255,44.137235
3,4,57.02,37.67,37.84,7.741266,7.740978,7.762811,7.796629,7.849472,7.935991,...,39.061605,39.623743,40.143299,40.666132,41.198445,41.715807,42.192246,42.641311,43.095348,43.595889
4,5,57.02,37.67,37.84,6.642168,6.647871,6.665882,6.686998,6.720952,6.786233,...,37.307641,38.113560,38.788978,39.330331,39.747478,40.085455,40.398576,40.749179,41.170187,41.662778


In [ ]:
# Create target_test_400 DataFrame
target_test_400 = df_test[['Sample_Index', 'Moi', 'NDF', 'Starch']].copy()

# Create spectra_test_400 DataFrame
# Select all columns except 'Moi', 'NDF', and 'Starch'
spectra_test_400 = df_test.drop(['Moi', 'NDF', 'Starch'], axis=1).copy()

print("Target DataFrame (target_test_400):")
display(target_test_400.head())

print("\nSpectra DataFrame (spectra_test_400):")
display(spectra_test_400.head())

Target DataFrame (target_test_400):


,Sample_Index,Moi,NDF,Starch
0,1,57.02,37.67,37.84
1,2,57.02,37.67,37.84
2,3,57.02,37.67,37.84
3,4,57.02,37.67,37.84
4,5,57.02,37.67,37.84



Spectra DataFrame (spectra_test_400):


,Sample_Index,3921.568654,3935.18188,3948.795106,3962.408332,3976.021558,3989.634784,4003.24801,4016.861236,4030.474462,...,7284.03545,7297.648676,7311.261902,7324.875128,7338.488353,7352.101579,7365.714805,7379.328031,7392.941257,7406.554483
0,1,7.209724,7.202985,7.208119,7.222696,7.258278,7.331684,7.448954,7.617220,7.838484,...,38.932550,39.595226,40.219869,40.823632,41.396258,41.917661,42.375544,42.786740,43.177964,43.578847
1,2,6.802202,6.786788,6.780681,6.782455,6.805124,6.866235,6.970652,7.124253,7.327331,...,37.986222,38.679829,39.260441,39.754916,40.186044,40.582608,40.971723,41.394751,41.882680,42.451031
2,3,7.890556,7.886369,7.891463,7.899888,7.926916,7.996553,8.118622,8.304178,8.555945,...,40.293261,40.944037,41.545459,42.118273,42.643149,43.082208,43.410055,43.651383,43.865255,44.137235
3,4,7.741266,7.740978,7.762811,7.796629,7.849472,7.935991,8.058430,8.225489,8.443056,...,39.061605,39.623743,40.143299,40.666132,41.198445,41.715807,42.192246,42.641311,43.095348,43.595889
4,5,6.642168,6.647871,6.665882,6.686998,6.720952,6.786233,6.889498,7.039912,7.239455,...,37.307641,38.113560,38.788978,39.330331,39.747478,40.085455,40.398576,40.749179,41.170187,41.662778


In [ ]:
import os

# Define the folder path in Google Drive
folder_path = '/content/drive/MyDrive/siware_data_avg_by_scannerID'

# Create the folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

# Define the file paths for saving the DataFrames with "_no_avg" suffix
df_test_path = os.path.join(folder_path, 'df_test_no_avg.csv')
target_test_400_path = os.path.join(folder_path, 'target_test_400_no_avg.csv')
spectra_test_400_path = os.path.join(folder_path, 'spectra_test_400_no_avg.csv')

# Save the DataFrames to CSV files
df_test.to_csv(df_test_path, index=False)
target_test_400.to_csv(target_test_400_path, index=False)
spectra_test_400.to_csv(spectra_test_400_path, index=False)

print(f"DataFrames saved to: {folder_path}")

DataFrames saved to: /content/drive/MyDrive/siware_data_avg_by_scannerID
